# 1.2、深度学习入门修炼手册：基于PyTorch的神经网络

## 一、基于PyTorch的单层感知机

### 1.1 搭建线性层

In [1]:
import torch
import torch.nn as nn

# 生成一批数据
B, C_in, C_out = 7, 256, 1024
x = torch.randn(B, C_in)

# 定义一层线性层
linear_layer = nn.Linear(in_features=C_in, out_features=C_out, bias=True)

# 打印线性层的权重矩阵的shape，即数据的维度
print("权重矩阵的shape: ", linear_layer.weight.shape)

# 打印线性层的偏置向量的shape
print("偏置矩阵的shape: ", linear_layer.bias.shape)

# 线性映射
z = linear_layer(x)

# 打印线性输出的shape
print(z.shape)


权重矩阵的shape:  torch.Size([1024, 256])
偏置矩阵的shape:  torch.Size([1024])
torch.Size([7, 1024])


In [2]:
print(linear_layer.weight.requires_grad)
print(linear_layer.bias.requires_grad)

True
True


### 1.2 搭建非线性激活函数

随后，我们在线性层之后接一层非线性激活函数，这里使用到PyTorch框架提供的 `nn.Sigmoid` 类来定义 `sigmoid` 激活函数层，代码如下所示：

In [3]:
# 定义Sigmoid激活函数
activation = nn.Sigmoid()

# 非线性激活输出
y = activation(z)

# 打印非线性输出的shape
print(y.shape)

torch.Size([7, 1024])


另外，除了这种定义的方式，我们还可以使用torch.nn.functional.sigmoid函数来实现同样的sigmoid函数功能，如下所示，我们将两种方法的结果做减法，然后使用sum操作求和，如果输出是0，表明两种方法是等价的。

In [4]:
import torch.nn.functional as F

y2 = F.sigmoid(z)
diff = y - y2
print(diff.sum().item())

0.0


### 1.3 搭建单层感知机


In [5]:
# 标准的PyTorch规范的神经网络模型搭建
class SingleLayerPerceptron(nn.Module):
    def __init__(self, in_dim, out_dim) -> None:
        super().__init__()
        self.layer = nn.Linear(in_features=in_dim, out_features=out_dim, bias=True)
        self.act = nn.Sigmoid()

    def forward(self, x):
        z = self.layer(x)
        y = self.act(z)

        return y

In [6]:
# 基于上方的单层感知机类，来构建一个模型
model = SingleLayerPerceptron(in_dim=256, out_dim=1024)

# 模型前向推理
y = model(x)

# 查看输出的shape
print(y.shape)

torch.Size([7, 1024])


## 二、基于PyTorch的多层感知机

### 2.1 搭建多层感知机

下方的代码给出了一个简单的三层感知机的PyTorch代码

In [7]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, in_dim, inter_dim, out_dim) -> None:
        super().__init__()
        self.layer = nn.Sequential(
            # 第一层感知机
            nn.Linear(in_features=in_dim, out_features=inter_dim, bias=True),
            nn.Sigmoid(),
            # 第二层感知机
            nn.Linear(in_features=inter_dim, out_features=inter_dim, bias=True),
            nn.Sigmoid(),
            # 第三层感知机
            nn.Linear(in_features=inter_dim, out_features=out_dim, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        y = self.layer(x)

        return y

在上方的代码中，我们用到了nn.Sequential类， 该类的作用是可以把传进内部的神经网络层串联在一起，我们只需要传进去一个数据数据，它会自行将输入数据从第一层一直传到最后一层，然后输出，注意，为了确保该类能正常运行，需要保证内部的层中，前一层的输出接口能有效对上后一层的输入接口，否则，会出现报错。比如，如果第一层需要需要处理输入的x，输出y，而第二层需要处理输入的x1和x2，但由于第一层只有一个y输出来，并流进第二层，那么第二层就会缺少一个有效的输入，便会导致报错。

In [8]:
# 基于上方的多层感知机类，来构建一个模型
model = MultiLayerPerceptron(in_dim=C_in, inter_dim=2048, out_dim=C_out)

# 模型前向推理
y = model(x)

# 打印输出的shape
print(y.shape)


torch.Size([7, 1024])


### 2.2 计算训练损失

搭建玩感知机模型，了解了前向推理的代码，接下来便可以去计算模型的损失，以便我们后续去训练这个模型。我们采用上方的多层感知机模型，并随机生成一组数据，如下所示：

In [10]:
# 生成一批数据
B, C_in, C_out = 3, 256, 1
x = torch.randn(B, C_in)

# 基于上方的多层感知机类，来构建一个模型
model = MultiLayerPerceptron(in_dim=C_in, inter_dim=2048, out_dim=C_out)

# 模型前向推理
y = model(x)

# 打印输出的shape
print(y.shape)
print(y)

torch.Size([3, 1])
tensor([[0.5079],
        [0.5075],
        [0.5135]], grad_fn=<SigmoidBackward0>)


在上方的代码中，我们省略了多层感知机的代码，以节省篇幅。我们定义了一组新的数据，其中数据的数量 $B$ 是 3，输入数据的特征维度 $C_{in}$ 是256，感知机的最终输出维度 $C_{out}$ 是 1，因为我们这里要模拟一个针对二分类任务的损失计算。

运行上方代码后，我们会得到输出 $y$ 的shape是形如[B, 1] 的。然后，我们随机定义了一组仅包含0和1的标签，其 `shape` 与模型输出的 `shape` 保持一致，接着，调用 PyTorch 提供的 `nn.BCELoss` 类来定义用于计算二元交叉熵的 `criterion` 变量 。

In [11]:
# 定义标签
target = torch.randint(low=0, high=2, size=[B, 1]).float()
print("标签: ", target)

# 定义二元交叉熵损失函数
criterion = nn.BCELoss(reduction="none")

# 计算二元交叉熵损失
loss = criterion(y, target)
print("二元交叉熵损失：", loss)
print("二元交叉熵损失的shape：", loss.shape)

标签:  tensor([[0.],
        [1.],
        [0.]])
二元交叉熵损失： tensor([[0.7090],
        [0.6783],
        [0.7205]], grad_fn=<BinaryCrossEntropyBackward0>)
二元交叉熵损失的shape： torch.Size([3, 1])


在上方代码中，nn.BCELoss类中的reduction参数被设置为none，表明我们不需要该来在计算完后对输出做任何处理。如果我们需要该类会把所有样本的损失加在一起，可以将reduction设置为sum；如果需要将所有样本的损失做个平均，则设置为mean，感兴趣的读者可以自行调试。

我们运行代码，即可看到计算出来的二元交叉熵损失和shape，下方给出了笔者的输出示例，由于输入数据和标签都是随机生成的，读者的结果可能会与笔者的有些不同。

运行上方的代码，即可看到结果均为0的输出，如下所示，表明二者的计算过程是一致的。

另外，在上方实现的多层感知机模型中，最后输出会被定义的sigmoid函数做处理，将其映射到0~1范围内，然后再去计算二元交叉熵，这是因为后者需要输入的数值在0~1的值域内。然而，由于sigmoid函数中的指数函数可能会造成某些计算的不稳定性，PyTorch额外提供了另一种计算二元交叉熵的nn.BCEWithLogitsLoss类和 binary_cross_entropy_with_logits函数来计算损失（二者也是等价的），对于这两个方法，我们就不需要在外部做sigmoid激活操作，内部会以等价的、但更稳定的计算方式来处理，并计算损失。

为了测试两种计算二元交叉熵的计算方法，我们编写了如下所示的代码，其中，criterion便是我们此前定义的nn.BCELoss类，要求输入的预测已被sigmoid函数处理过，criterion2则是 nn.BCEWithLogitsLoss类，无需我们外部对预测值进行Sigmoid操作。

In [ ]:
criterion2 = nn.BCEWithLogitsLoss(reduction="none")
y = torch.tensor([-2, 3, 4]).float()
target = torch.tensor([1, 0, 1]).float()
loss1 = criterion(F.sigmoid(y), target)
print(loss1)
loss2 = criterion2(y, target)
print(loss2)
loss3 = F.binary_cross_entropy_with_logits(y, target, reduction="none")
print(loss3)

tensor([2.1269, 3.0486, 0.0181])
tensor([2.1269, 3.0486, 0.0181])
tensor([2.1269, 3.0486, 0.0181])


运行上方的代码，我们即可看到完全相等的输出，如下所示，表明三者的操作完全是等价。

因此，通常情况下， 我们往往会使用nn.BCEWithLogitsLoss类来定义二元交叉熵函数，避免运算的过程中出现恼人的NAN问题，当然，在有些时候，我们也会用nn.BCELoss类来处理已经被映射到0~1范围内的数值，二者并不是一个完全A取代B的关系，需要我们自己根据具体情况来做具体的选择。